In [107]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [108]:
retrieved_data = requests.get(**{
    "url": "https://api.pushshift.io/reddit/submission/search/",
    "params": {
        'after': 1585699199, #get dates after March 31, 2020
        'before': 1588291200, #get dates before May 1, 2020
        'subreddit': 'dlsu',
        'size': 500,
        'fields': ["author", "subreddit", "created_utc", "num_comments", "score", "title", "selftext"]
    }
}).json()["data"]

In [111]:
def format_data():
    data = {
        "author": [],
        "subreddit": [],
        "created_at": [],
        "num_of_comments": [],
        "score": [],
        "title": [],
        "description": []
    }
    for reddit in retrieved_data:
        data["author"].append(reddit["author"])
        data["subreddit"].append(reddit["subreddit"])
        data["title"].append(reddit["title"])
        data["description"].append(reddit["selftext"])
        data["score"].append(reddit["score"])
        data["num_of_comments"].append(reddit["num_comments"])
        data["created_at"].append(reddit["created_utc"])
    return data

df["created_at"] = pd.to_datetime(df["created_at"],unit="s",origin="unix")
df["day_of_week"] = df["created_at"].dt.day_name()

In [112]:
total_submission_cnt = len(df)
text1 = f"Total Submission Count: {total_submission_cnt}"
print(text1)
author_with_most_submissions = df["author"].mode()[0]
author_submission_count = len(df[df["author"] == author_with_most_submissions])
text2 = f"Author with Most Submissions: {author_with_most_submissions} ({author_submission_count} submissions)"
print(text2)
submission_with_highest_score = df.loc[df["score"].idxmax()]
text3 = f"Submission with highest score: '{submission_with_highest_score['title']}' by {submission_with_highest_score['author']} ({submission_with_highest_score['score']} submissions)"
print(text3)
submission_with_highest_comms = df.loc[df["num_of_comments"].idxmax()]
text4 = f"Submission with highest comments: '{submission_with_highest_comms['title']}' by {submission_with_highest_comms['author']} ({submission_with_highest_comms['num_of_comments']} comments)"
print(text4)
df_agg = df.groupby(['day_of_week'])["day_of_week"].count()
max_day_index = df_agg.idxmax()
text5 = f"Day of week with most submissions {max_day_index} ({df_agg.loc[df_agg.idxmax()]} submissions)"
print(text5)


Total Submission Count: 100
Author with Most Submissions: camnacio_13 (11 submissions)
Submission with highest score: 'Opportunities in abroad' by Cheezyreine (9 submissions)
Submission with highest comments: 'Laptop' by vgtrxx (14 comments)
Day of week with most submissions Sunday (18 submissions)
